In [7]:
import openai

client = openai.OpenAI()

PROMPT = """
I have the following functions in my system.

`get_weather`
`get_currency`
`get_news`

All of them receive the name of a country as an argument ie. (get_news("Spain"))

Please answer with the name of the function that you would like to call.

Please say nothing else, just the name of the function with the arguments.

Answer the following question:

What is the weather in Greece?
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": PROMPT},
    ],
)

message = response.choices[0].message.content
print(message)


get_weather("Greece")


In [5]:
for choice in response.choices:
    print(choice)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))
